In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
# 한글 설정
# pip install koreanize_matplotlib
plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False)
# root = 'C:/workspace/python/project/data/'
root = 'G:/workspace/python/python_project/data/'

# 구글드라이브 : https://drive.google.com/drive/folders/1zIzm1o8-3uxcWSU2DoWpB8aV0Oxdfz_P?usp=sharing

In [ ]:
'''
건축거래 현황 데이터 프레임
'''
house_transaction_df = pd.read_csv(root + '주거실태_Data/행정구역별_건축물거래현황_20241126100629.csv', encoding='cp949', header=None)
# house_transaction_df = pd.read_csv(root + '주거실태_Data/행정구역별_건축물거래현황_20241126100629.csv', encoding='cp949')

In [ ]:
house_transaction_df.loc[:, 0:5][:5]

In [ ]:
house_transaction_df.columns = house_transaction_df.iloc[0]
house_transaction_df

In [ ]:
house_transaction_df = house_transaction_df.rename(columns={'행정구역별(2)':'구'})
house_transaction_df

In [ ]:
house_transaction_df = house_transaction_df.drop(columns=['행정구역별(1)'])
house_transaction_df

In [ ]:
'''
0 행 삭제 클래스
'''
class ManagedDataFrame:
    def __init__(self, name, df):
        self.name = name
        self.df = df
        self.row_deleted = False  # 플래그 초기화

    def delete_row_once(self):
        if not self.row_deleted and 0 in self.df.index:
            self.df = self.df.drop(0)
            self.row_deleted = True
            print(f"{self.name}: 0행이 삭제되었습니다.")
        else:
            print(f"{self.name}: 삭제 작업이 이미 완료되었거나 0행이 없습니다.")

    # 객체 출력 시 사용자 정의 내용 반환
    def __repr__(self):
        return f"ManagedDataFrame(name={self.name}, row_deleted={self.row_deleted}, df=\n{self.df}\n)"

In [ ]:
transaction_df = ManagedDataFrame('house_transaction_df', house_transaction_df)
transaction_df.delete_row_once()

In [ ]:
transaction_df = transaction_df.df
print(transaction_df.dtypes)
transaction_df

In [ ]:
# transaction_df.to_csv(root + '주거실태_Data/행정구역별_건축물거래현황.csv', encoding='cp949', index=False)
transaction_df

In [ ]:
# DataFrame 생성
df = transaction_df.copy()
df.iloc[:, 1:]

# '년도', '월' 열을 추가하고 기존의 '동(호)수'와 '면적' 정보를 각각 새롭게 정리
df_melted = pd.melt(df, id_vars=['구'], value_vars=(df.iloc[:, 1:]), var_name='년도_월', value_name='값')
df_melted[df_melted['구']=='종로구']

In [ ]:
df.columns

In [ ]:
tmp_df = df.iloc[:, 1:]
tmp_df.columns

In [ ]:
category_2021 = df.columns.str.contains('2021.01')
category_2021[0] = True
category_2021

In [ ]:
category_2021_df = df.loc[:, category_2021]
category_2021_df
col_2021 = category_2021_df.loc[1].values
category_2021_df.columns = col_2021
display(category_2021_df[:3])

In [ ]:
df_2021 = pd.DataFrame(category_2021_df)
df_2021['년월'] = '2021.01'

In [ ]:
df_2021 = df_2021.drop(1)

In [ ]:
# 데이터프레임 병합
df_combined = pd.concat([df_2021], ignore_index=True)
# 결과 확인
display(df_combined)

In [ ]:
'''   (여기서 부터 작업함 -- 희만 --)
건축거래 현황 데이터 프레임
'''
house_transaction_df = pd.read_csv(root + '주거실태_Data/행정구역별_건축물거래현황_20241126100629.csv', encoding='cp949', header=None)
# house_transaction_df = pd.read_csv(root + '주거실태_Data/행정구역별_건축물거래현황_20241126100629.csv', encoding='cp949')

In [ ]:
house_transaction_df

In [ ]:
def precleaning_transaction(df, year) :
  copy_df = df.copy()
  copy_df.columns = copy_df.iloc[0] + '_' + copy_df.iloc[1]
  copy_df = copy_df.iloc[2:, 1:]
  copy_df = copy_df.rename(columns = {'행정구역별(2)_행정구역별(2)' : '구'})

  condition1 = copy_df.columns.str.contains(year)
  select_columns = copy_df.loc[:, condition1].columns
  columns = []
  columns.append('구')
  for column in select_columns :
    columns.append(column)
  
  selected_df = copy_df[columns]
  # Melt 함수를 사용하여 열을 행으로 변환
  df_melted = selected_df.melt(id_vars=['구'], var_name='월', value_name='값')
  
  # 월별 동(호)수와 면적을 구분하기 위해 컬럼을 분리
  df_melted[['연도_월', '항목']] = df_melted['월'].str.split('_', expand=True)
  
  # 항목이 동(호)수인지 면적인지에 따라 값 분리
  df_melted = df_melted.pivot_table(index=['구', '연도_월'], columns='항목', values='값', aggfunc='first').reset_index()
  return df_melted

In [ ]:
house_transaction_2021_df = precleaning_transaction(house_transaction_df, '2021')
house_transaction_2021_df[:5]
house_transaction_2021_df['년도'] = 2021
house_transaction_2021_df[:5]

In [ ]:
house_transaction_2022_df = precleaning_transaction(house_transaction_df, '2022')
house_transaction_2022_df[:5]
house_transaction_2022_df['년도'] = 2022
house_transaction_2022_df[:5]

In [ ]:
house_transaction_2023_df = precleaning_transaction(house_transaction_df, '2023')
house_transaction_2023_df[:5]
house_transaction_2023_df['년도'] = 2023
house_transaction_2023_df[:5]

In [ ]:
# 데이터프레임 병합
house_transaction_combined = pd.concat([house_transaction_2021_df, house_transaction_2022_df, house_transaction_2023_df], ignore_index=True)
# 결과 확인
print(house_transaction_combined)

In [ ]:
house_transaction_combined.isna().sum()

In [ ]:
house_transaction_combined = house_transaction_combined.rename(columns={'동(호)수 (동(호)수)':'동(호)수'})

In [ ]:
house_transaction_combined['동(호)수']    = house_transaction_combined['동(호)수'].astype(int)
house_transaction_combined['면적 (천㎡)'] = house_transaction_combined['면적 (천㎡)'].astype(int)

In [ ]:
house_transaction_grouped = house_transaction_combined.groupby(['년도', '구']).agg({'동(호)수': 'sum',
                                                                                    '면적 (천㎡)': 'sum'}).reset_index()
house_transaction_grouped

In [ ]:
# 데이터 준비
grouped_data = house_transaction_grouped.copy()
grouped_data = grouped_data[grouped_data['구'] != '소계']

In [ ]:
# 연도별 동(호)수 비교 막대 그래프
plt.figure(figsize=(12, 6))
sns.barplot(x='년도', y='동(호)수', hue='구', data=grouped_data, palette='tab10')
plt.title('연도별 구별 동(호)수', fontsize=16)
plt.ylabel('동(호)수', fontsize=12)
plt.xlabel('년도', fontsize=12)
plt.legend(title='구', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# 동(호)수와 면적 (천㎡)의 상관 관계
plt.figure(figsize=(8, 6))
sns.scatterplot(x='동(호)수', y='면적 (천㎡)', hue='년도', style='구', data=grouped_data, palette='viridis', s=100)
plt.title('동(호)수와 면적 (천㎡)의 관계', fontsize=16)
plt.xlabel('동(호)수', fontsize=12)
plt.ylabel('면적 (천㎡)', fontsize=12)
plt.legend(title='년도/구', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# 연도별 면적 (천㎡) 변화 추세선
plt.figure(figsize=(12, 6))
sns.lineplot(x='년도', y='면적 (천㎡)', hue='구', data=grouped_data, marker='o', palette='tab20')
plt.title('연도별 구별 면적 (천㎡) 변화', fontsize=16)
plt.ylabel('면적 (천㎡)', fontsize=12)
plt.xlabel('년도', fontsize=12)
plt.legend(title='구', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
# 데이터 추출
# house_transaction_grouped
# 소계 데이터 제외
# grouped_data
# 사용할 것